# Calculate area
Calculate the area of each fire severity class for all wildfires and prescribed burns from 2005-2020 in the Northern Jarrah Forest, Western Australia (calculate the areas in Table 3 from Dixon et al., 2022) 

In [12]:
import ee
ee.Initialize() #be sure to run earthengine authenticate first
import pandas as pd
import geopandas as gpd
import os
import warnings
warnings.filterwarnings('ignore')

In [8]:
# read the fire perimeter dataset (this is essentially DBCA-060 with a custom fire ID associated with each fire severity prediction)
gdf = gpd.read_file("data/vector/DBCAFireHistory_2005_2020_gte50ha.shp")
# we're going to map over each unique fire ID to put those in a list
fires = list(gdf['fireID'].unique())
# create a variable for the ee image collection with predicted fire severity data
imcol = ee.ImageCollection('projects/euc-fire-sw-2022/assets/severity_pred_forested')
# this imcol dataset has already been processed to remove non-forestest areas according to Hansen global forest cover

In [9]:
# create a function that returns the area of each predicted fire severity class given the fire ID
def get_severity_by_class(ID):
    
    # reference imcol from previous cell
    img = imcol.filterMetadata('ID', 'equals', ID).first()
    
    # add an area band 
    area_img = ee.Image.pixelArea().divide(1e6) # original pixelArea band is in m2, have to convert to km2
    # if you want hectares, replace 1e6 with 10000
    # add that band to out img
    img = img.addBands(area_img)
    
    # creating five masks and doing it separately to learn how the calculation works
    c1_mask = img.select('f_severity').eq(1) 
    c1_area = img.updateMask(c1_mask).select('area').rename('c1_area') # class 1 area in km2

    c2_mask = img.select('f_severity').eq(2)
    c2_area = img.updateMask(c2_mask).select('area').rename('c2_area') # class 2 area in km2

    c3_mask = img.select('f_severity').eq(3)
    c3_area = img.updateMask(c3_mask).select('area').rename('c3_area') # class 3 area in km2

    c4_mask = img.select('f_severity').eq(4)
    c4_area = img.updateMask(c4_mask).select('area').rename('c4_area') # class 4 area in km2

    c5_mask = img.select('f_severity').eq(5)
    c5_area = img.updateMask(c5_mask).select('area').rename('c5_area') # class 5 area in km2
    
    # we're going to use the fire polygon to clip the raster to make sure we only quantify pixels within the 
    # fire polygon boundary
    fire_poly = ee.FeatureCollection('projects/euc-fire-sw-2022/assets/data/DBCAFireHistory_2005_2020_gte50ha')\
          .filterMetadata('fireID','equals',f).first()
    
    # add the area bands for each severity class and then select only those
    img = img.addBands([c1_area, c2_area, c3_area, c4_area, c5_area])
    img = img.select(['c1_area', 'c2_area', 'c3_area', 'c4_area', 'c5_area'])
    
    # we're calling .getInfo() beware!! 
    area_sum = img.reduceRegion(reducer=ee.Reducer.sum(),geometry=fire_poly.geometry(),scale=30).getInfo()
    
    # putting our results into a pandas df
    area_df = pd.DataFrame([area_sum])
    area_df['f'] = f
    return area_df


# running this block should take a couple of minutes
l = [] # make an empty list
for f in fires:
    try:
        result = get_severity_by_class(f) # return the result to our list
        l.append(result)
        #print(f, 'completed') # print or not you decide
    except:
        print(f, 'no prediction') # we don't have a prediction for a few fires


0279_f no prediction
0610_f no prediction
0909_f no prediction
5325_f no prediction
6408_f no prediction


In [16]:
# we are going in to the modis dates dataset to tell us which was adjusted with modis and which was not
df_dates = pd.read_csv("data/tables/all_fires_dates-start-end.csv")
df_dates['date_adj'].unique()
det = df_dates[df_dates['date_adj'].isin(['modis_1k', 'modis_250m', 'manual'])]
det['modis_det'] = 'modis'
ndet = df_dates[df_dates['date_adj'].isin(['no_adj'])]
ndet['modis_det'] = 'no_modis'
c = pd.concat([det, ndet])
# create two dictionaries -- one with modis detected and another with fire type (WF or PB)
modis_dict = dict(zip(c['fireID'], c['modis_det']))
type_dict = dict(zip(c['fireID'], c['fih_fire_t'])) 

In [19]:
# put together the results from get_severity_by_class()
df2 = pd.concat(l)
df2['modis'] = df2['f'].map(modis_dict) # apply the modis and type dicts to group by those variables
df2['type'] = df2['f'].map(type_dict)
final_area = df2.groupby(['modis','type'])['c1_area','c2_area','c3_area','c4_area','c5_area'].sum()
final_area
# below should be Table 3 in the paper

c1_area      c2_area      c3_area     c4_area     c5_area
modis    type                                                               
modis    PB    1817.642568  2504.314715  2116.093765  900.343031  203.380862
         WF      66.688452   103.583322   419.820752  566.412887  682.562411
no_modis PB     266.855397   157.738154    70.544681   16.544002    1.593697
         WF       2.712354     2.508251     7.829166    5.132283    3.451964